# Demo: Exploratory Execution

We start this tutorial from scratch in blank notebooks to whet your appetite for knowing the openTEPES.

## 1. Installation

**[Recommendation]** If you are running this tutorial in Binder [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/IIT-EnergySystemModels/openTEPES-tutorial/HEAD), ignore the installation step.

In case, you prefer use your personal computer, the installation of the openTEPES model and its dependencies, please follow the section **installation** of the [README](https://github.com/IIT-EnergySystemModels/openTEPES#readme).

## 2. Quick-run

As a first step, we import the function ``openTEPES_run(DIR, CaseName, SolverName, pIndLogConsole)`` as:

In [2]:
from openTEPES.openTEPES import openTEPES_run

The function returns the model as a pyomo object containing all model elements (i.e., set, param, var, etc.).
Then, we only need to specify the path where is the case (DIR), the case name (CaseName), the solver name (SolverName), and the parameter (i.e., ``Yes`` or ``No``) if you want to write log information in the console (IndLogConsole).
For our example, we don't specify the path because the case folder is in the same place as this notebook.

Then, we created a folder with the name of our case study (e.g., ``9n`` in our case), where are all the input files needed.
And, we use the ``HiGHS`` solver that is already installed if you are running this tutorial in Binder.

In [3]:
DIR =''
CaseName = '9n'
SolverName = 'glpk'
IndOutputResults = 'Yes'
IndLogConsole = 'No'

In [4]:
model = openTEPES_run(DIR, CaseName, SolverName, IndOutputResults, IndLogConsole)

Input data                             ****
Reading    input data                  ...  0 s


C:\Users\erik_\miniconda3\lib\site-packages\pyomo\core\base\block.py:541: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.add_component(name, val)


Setting up input data                  ...  6 s
Setting up variables                   ...  2 s
Investment           model formulation ****
Generation operation model formulation ****
Network    switching model formulation ****
Network    operation model formulation ****
Writing LP file                        ...  14 s
Problem solving                        ****
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x428081
  Lower bound: 142.15017491480492
  Upper bound: 142.15017491480492
  Number of objectives: 1
  Number of constraints: 214031
  Number of variables: 272626
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 272626
  Number of nonzer

The output files can be found in the folder called ``9n`` , both the ``csv`` and ``html`` files.
As example of it, we read the total output per technology as follows:

### Reading the CSV file TechnologyOutput as a pandas dataframe

In [5]:
import pandas as pd
df = pd.read_csv(CaseName+'/oT_Result_TechnologyGeneration_'+CaseName+'.csv')
df.head()

,Scenario,Period,LoadLevel,Coal,ESS,Gas,Nuclear,Oil,RES
0,sc01,y2030,2030-01-01T03:00:00+01:00,0.0,0.000000,0.000000,771.6,0.0,241.191319
1,sc01,y2030,2030-01-01T07:00:00+01:00,0.0,0.000000,0.000000,771.6,0.0,109.232531
2,sc01,y2030,2030-01-01T11:00:00+01:00,0.0,0.000000,224.004108,771.6,0.0,108.043994
3,sc01,y2030,2030-01-01T15:00:00+01:00,0.0,0.000000,224.004108,771.6,0.0,142.577969
4,sc01,y2030,2030-01-01T19:00:00+01:00,0.0,29.765856,233.222947,771.6,0.0,78.910838


Notice that the results are in ``tetra-hourly`` resolution since we have the ``TimeStep`` equal to ``4`` in the CSV file ``oT_Data_Parameter_9n``.
Henceforth, you can play with the ``TimeStep`` by changing it from ``1`` (hourly) to ``4`` (tetra-hourly), and check the results.

### Reading the plot TechnologyOutput

In [6]:
from IPython.display import IFrame
IFrame(src=CaseName+'/oT_Plot_TechnologyGeneration_2030_sc01_'+CaseName+'.html', width=1400, height=650)

## 3. Running Step by Step

The running step by step of the openTEPES model can be found here:

- [1.3-openTEPES-Running-StepByStep.ipynb](1.3-openTEPES-Running-StepByStep.ipynb)